### Módulos

In [47]:
import csv
import re
from datetime import datetime, time
import uuid
import os

### Conversión formato

In [48]:
time_range_pattern = re.compile(r"^\s*(?P<h1>\d{1,2})h(?P<m1>\d{2})'\s*-\s*(?P<h2>\d{1,2})h(?P<m2>\d{2})'\s*$")

def parse_time(time_str, date_str):
    date_obj = datetime.strptime(date_str, "%d/%m/%y").date()
    m = time_range_pattern.match(time_str)
    if not m:
        raise ValueError(f"Time range string not in expected format: {time_str!r}")
    
    # Extract and convert
    h1 = int(m.group('h1'))
    m1 = int(m.group('m1'))
    h2 = int(m.group('h2'))
    m2 = int(m.group('m2'))
    
    # Build time objects (will raise ValueError if out-of-range)
    t1 = time(hour=h1, minute=m1)
    t2 = time(hour=h2, minute=m2)
    
    # Combine with the provided date
    start_dt = datetime.combine(date_obj, t1).strftime("%Y%m%dT%H%M%SZ")
    end_dt = datetime.combine(date_obj, t2).strftime("%Y%m%dT%H%M%SZ")
    
    return start_dt, end_dt

### Apertura de archivo

In [49]:
def parse_class_schedule(file_path, subjectName):
    classes = []
    
    with open(file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=';')
        found_header = False
        for row in reader:

            if not found_header:
                if len(row) > 6 and row[1] == "Grupo" and row[2] == "Clase nº":
                    found_header = True
                continue

            # A valid class row contains "Clase", "Grupo", "Día", etc.
            if any("Clase" in cell for cell in row) and "Listado de clases" not in row[0]:
                # Extract only relevant parts
                grupo = row[1].strip()
                fecha = row[5].strip()
                hora = row[6].strip()
                aula = row[7].strip()

                inicio, fin = parse_time(hora, fecha)

                class_entry = {
                    "subjectName": subjectName,
                    "groupName": grupo,
                    "beginDate": inicio,
                    "endDate": fin,
                    "location": aula
                }
                
                classes.append(class_entry)
    
    return classes

### Generación eventos

Abre un archivo proporcionado, sin destruirlo, y añade el evento de los parámetros

In [50]:
def generate_event(file, subjectName, groupName, beginDate, endDate, location):
    timestamp = datetime.now().strftime("%Y%m%dT%H%M%SZ")
    event_uid = str(uuid.uuid4())

    with open(file, "a") as f:
        f.write("BEGIN:VEVENT\n")
        f.write(f"UID:{event_uid}\n")
        f.write(f"DTSTAMP:{timestamp}\n")
        f.write(f"CREATED:{timestamp}\n")
        f.write(f"LAST-MODIFIED:{timestamp}\n")
        f.write(f"DTSTART;TZID=Europe/Madrid:{beginDate}\n")
        f.write(f"DTEND;TZID=Europe/Madrid:{endDate}\n")
        f.write("TRANSP:OPAQUE\n")
        f.write(f"SUMMARY:{subjectName} - {groupName}\n")
        f.write(f"LOCATION:{location}\n")
        f.write("SEQUENCE:0\n")
        f.write("END:VEVENT\n")

### Bucle principal

In [ ]:

current_dir = os.getcwd()

files = [f for f in os.listdir(current_dir) if os.path.isfile(os.path.join(current_dir, f)) and os.path.abspath(os.path.join(current_dir, f)) not in [os.path.join("Notebook1.ipynb")]]


for file in files:
    subjectName = re.match(r"^(.*?)_", file)
    if subjectName:
        subjectName = subjectName.group(1)
        input_file = file
        output_file = f"{current_dir}/{subjectName}.ics"

        CE = "CE"+str(input(f"\nCuál es el grupo de CE para {subjectName}? Si no hay, deja en blanco\n"))
        PA = "PA"+str(input(f"\nCuál es el grupo de PA para {subjectName}? Si no hay, deja en blanco\n"))
        TG = "TG"+str(input(f"\nCuál es el grupo de TG para {subjectName}? Si no hay, deja en blanco\n"))
        PL = "PL"+str(input(f"\nCuál es el grupo de PL para {subjectName}? Si no hay, deja en blanco\n"))

        groups = [CE, PA, TG, PL]

        classes = parse_class_schedule(input_file, subjectName)

        with open(output_file, "w") as f:
            f.write("BEGIN:VCALENDAR\n")
            f.write("VERSION:2.0\n")
            f.write("CALSCALE:GREGORIAN\n")
            f.write("METHOD:PUBLISH\n")
            f.write("PRODID:-//lucasberredo//EN\n")
            f.write("X-WR-CALNAME: Horario 24-25\n")

        for event in classes:
            if event["groupName"] in groups:
                generate_event(output_file, event["subjectName"], event["groupName"], event["beginDate"], event["endDate"], event["location"])

        with open(output_file, "a") as f:
            f.write("END:VCALENDAR\n")

Elec
